## DPR + Reranking

In [ ]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoder, DPRContextEncoderTokenizer
import torch, faiss, numpy as np
from sentence_transformers import CrossEncoder
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain.schema import Document

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ctx_model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base").to(device)

q_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
q_model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base").to(device)

cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [ ]:
class DPRVectorStore:
    def __init__(self, ctx_model, q_model, ctx_tokenizer, q_tokenizer):
        self.ctx_model = ctx_model
        self.q_model = q_model
        self.ctx_tokenizer = ctx_tokenizer
        self.q_tokenizer = q_tokenizer
        self.index = None
        self.passages = []
        self.embeddings = None

    def build_index(self, passages):
        self.passages = passages
        inputs = self.ctx_tokenizer(passages, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            self.embeddings = self.ctx_model(**inputs.to(device)).pooler_output.cpu().numpy()
        dim = self.embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dim)
        self.index.add(self.embeddings)

    def query(self, question, top_k=5):
        q_inputs = self.q_tokenizer(question, return_tensors="pt").to(device)
        with torch.no_grad():
            q_embed = self.q_model(**q_inputs).pooler_output.cpu().numpy()
        D, I = self.index.search(q_embed, top_k)
        results = [self.passages[i] for i in I[0]]
        return results

    def rerank(self, question, passages):
        pairs = [(question, p) for p in passages]
        scores = cross_encoder_model.predict(pairs)
        ranked = [p for _, p in sorted(zip(scores, passages), reverse=True)]
        return ranked, scores

    def save_index(self, path="dpr_index.faiss"):
        faiss.write_index(self.index, path)

    def load_index(self, path="dpr_index.faiss"):
        self.index = faiss.read_index(path)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../data/Sales/Case Studies/Aura Health.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

print("Number of chunks:", len(chunks))


Number of chunks: 19


In [ ]:
retriever = DPRVectorStore(ctx_model, q_model, ctx_tokenizer, q_tokenizer)
retriever.build_index([c.page_content for c in chunks])
print("Index built with", len(chunks), "chunks")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Index built with 19 chunks


In [ ]:
query = "What is Emotion Intelligence Integration?"

dpr_results = retriever.query(query, top_k=5)
reranked, scores = retriever.rerank(query, dpr_results)

print("DPR Top-k Results:")
for i, p in enumerate(dpr_results, 1):
    print(f"{i}. {p}")

print("\nCross-Encoder Reranked Results:")
for i, (p, s) in enumerate(zip(reranked, scores), 1):
    print(f"{i}. {p} (score: {s:.4f})")


DPR Top-k Results:
1. Emotion Detection Gap
Therapists and wellness coaches needed better tools to create, 
publish, and monetize their content, but the platform lacked an 
intuitive, scalable interface to support growing creator needs.
Creator Experience Limitations
Aura’s expansion into workplaces required robust infrastructure to 
handle enterprise-scale usage, real-time personalization, and
integration with collaboration tools all without sacriﬁcing
performance or privacy.
Scalability for Enterprise Use
2. Impact
Emotion AI delivered a 40% increase in content relevance, as users 
engaged more with mood-matched meditations, stories, and
therapy sessions.
Real-Time Personalization
Upgraded UI/UX and personalized content journeys led to a 35% 
uplift in daily active users and a 25% increase in session duration 
across platforms.
Boost in User Engagement
The creator tools resulted in a 60% increase in audio content 
ploads and a 3x growth in active wellness coaches contributing 
to the

In [90]:
from langgraph.prebuilt import ToolNode
from langchain.tools import tool

@tool
def retrieve_documents(query: str):
    """Retrieve relevant documents."""
    dpr_results = retriever.query(query, top_k=3)
    reranked, scores = retriever.rerank(query, dpr_results)

    return reranked

tools = [retrieve_documents]
tool_node = ToolNode(tools)

In [ ]:
EMAIL_SYSTEM_PROMPT = """
You are a professional email assistant for our company's sales team. Your role is to respond to customer inquiries using ONLY information from our knowledge base.

CRITICAL RULES:
1. You MUST respond in proper business email format with subject line, salutation, body, and signature
2. If the customer's question can be answered using the provided context, write a helpful, professional email response
3. If the information is NOT in the knowledge base (context shows "NO_RELEVANT_INFORMATION_FOUND"), respond with a polite email explaining this
4. Never invent information or use external knowledge
5. Maintain a professional, helpful tone in all communications
6. Format your response as a ready-to-send email
7. Always start the subject with "Re: " followed by the original subject or an appropriate title

EMAIL FORMAT:
Subject: Re: [Original Subject or Appropriate Title]

Dear [Customer Name],

[Professional email body acknowledging their query and providing information or explaining limitations]

[Clear next steps or contact information if needed]

Best regards,
[Sale Team]
[Strategisthub]
"""

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langgraph.graph import StateGraph, MessagesState
from langchain_core.messages import SystemMessage
from langgraph.graph import START, END
from langgraph.checkpoint.memory import MemorySaver

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind_tools(tools)

def call_model(state: MessagesState):
    response = model.invoke(
        [
            SystemMessage(
                        content=EMAIL_SYSTEM_PROMPT
                    )
        ]
        + state["messages"]
        )
    return {"messages": [response]}

def should_continue(state: MessagesState):
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tools"
    return END

# 7. Build graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# app = workflow.compile()
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [93]:
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Inquiry About Emotion Intelligence Integration

Dear [Customer Name],

Thank you for your inquiry regarding Emotion Intelligence Integration. 

Emotion Intelligence Integration involves the development and deployment of AI models that detect emotional cues from user interactions across various platforms such as Slack, Zoom, and mobile/web applications. This technology enables dynamic content recommendations tailored to users' real-time mental states, enhancing user engagement and content relevance.

For instance, the implementation of this integration has led to a 40% increase in content relevance, as users engage more with mood-matched meditations, stories, and therapy sessions. Additionally, it has contributed to a significant boost in user engagement metrics.

If you have any further questions or need more detailed information, please feel free to reach out.

Best regards,  
Sales Team  
Strategisthub


In [94]:
query = "What is aura health?"
config = {"configurable": {"thread_id": "1"}}
response = app.invoke(
    {"messages": [{"role": "user", "content": query}]},
    config
)
print(response["messages"][-1].content)

Subject: Re: Inquiry About Aura Health

Dear [Customer Name],

Thank you for your inquiry about Aura Health.

Aura Health is a globally recognized leader in mental wellness, trusted by over 8 million users. It has been honored with several awards, including Apple’s Best of Apps and the Very Well Mind Online Therapy and Wellness Award. Aura provides a category-defining platform in digital wellness, combining the expertise of the world’s best coaches and therapists to deliver a deeply personalized self-care journey for each user.

The platform's vision is to create a global digital ecosystem for mental health, offering healing anytime and anywhere. A key feature of Aura is its Emotion AI, which reads user emotions from their interactions across various tools and recommends tailored content such as guided meditations, sleep stories, and cognitive therapy sessions.

If you have any further questions or need additional information, please feel free to reach out.

Best regards,  
Sales Team 

# Testing Graph

In [8]:
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from supabase_auth import datetime
from langchain_core.documents import Document
from pathlib import Path
import json, shutil
import uvicorn
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import sys
import os

project_root = os.path.abspath("..")
sys.path.append(project_root)

from app.data_loader import read_uploaded_file

from app.data_loader import read_uploaded_file
from app.tools import create_retriever_tool
from app.graph_builder import build_workflow
import os
# from app.vectorstore_weaviate import create_or_load_vectorstore, load_vectorstore
from app.vectorstore_supabase import (
    get_active_prompt,
)


# def handle_query(request):
#     active_prompt_data = get_active_prompt()
#     if not active_prompt_data or "active_prompt" not in active_prompt_data:
#         # raise HTTPException(status_code=404, detail="No active prompt found.")
#         print("No active prompt found.")
#         return {"response": "No active prompt found."}
    
#     system_prompt = active_prompt_data["active_prompt"]["prompt"]
#     tools = create_retriever_tool()
#     graph = build_workflow(tools, system_prompt)
#     config = {"configurable": {"thread_id": "1"}}
#     response = graph.invoke({"messages": request.query}, config=config)
#     return {"response": response["messages"]}

In [12]:
# def handle_query(request):
#     active_prompt_data = get_active_prompt()
#     if not active_prompt_data or "active_prompt" not in active_prompt_data:
#         print("No active prompt found.")
#         return {"response": "No active prompt found.", "sources": []}
    
#     system_prompt = active_prompt_data["active_prompt"]["prompt"]
#     tools = create_retriever_tool()
#     graph = build_workflow(tools, system_prompt)
#     config = {"configurable": {"thread_id": "1"}}

#     result = graph.invoke({"messages": request.query}, config=config)

#     messages = result["messages"]

#     # ----- Extract final response -----
#     final_ai_msg = None
#     for msg in messages:
#         if msg.__class__.__name__ == "AIMessage" and msg.content:
#             final_ai_msg = msg.content

#     # ----- Extract sources from ToolMessage -----
#     sources = []
#     for msg in messages:
#         if msg.__class__.__name__ == "ToolMessage":
#             if hasattr(msg, "artifact") and msg.artifact:
#                 for item in msg.artifact:
#                     sources.append({
#                         "source": item["metadata"].get("source"),
#                         "content": item["page_content"],
#                         "rerank_score": item.get("rerank_score")
#                     })

#     return {
#         "response": final_ai_msg,
#         "sources": sources
#     }


def handle_query(request):
    active_prompt_data = get_active_prompt()
    if not active_prompt_data or "active_prompt" not in active_prompt_data:
        print("No active prompt found.")
        return {"response": "No active prompt found."}
    
    system_prompt = active_prompt_data["active_prompt"]["prompt"]
    tools = create_retriever_tool()
    graph = build_workflow(tools, system_prompt)
    config = {"configurable": {"thread_id": "1"}}

    result = graph.invoke({"messages": request.query}, config=config)
    messages = result["messages"]

    # ----- Extract final AI response -----
    final_ai_msg = ""
    for msg in messages:
        if msg.__class__.__name__ == "AIMessage" and msg.content:
            final_ai_msg = msg.content

    # ----- Extract sources from ToolMessage -----
    source_names = []
    for msg in messages:
        if msg.__class__.__name__ == "ToolMessage" and hasattr(msg, "artifact"):
            for item in msg.artifact:
                src = item["metadata"].get("source")
                if src and src not in source_names:
                    source_names.append(src)

    # ----- Append sources at the end of response -----
    if source_names:
        final_ai_msg += "\n\nSources:\n" + "\n".join(
            f"- {src}" for src in source_names
        )

    return {"response": final_ai_msg}


In [13]:
response = handle_query(type('obj', (object,), {'query': 'What is aura health?'}))
print(response)

I am in the tool...
Got some data...
Re-Ranking the results...
{'response': 'Aura Health is a leading global platform in the mental wellness industry, trusted by over 8 million users. It has been recognized as a "BEST OF APPS" winner by Apple and received the Verywell Mind Online Therapy and Wellness Award in 2023. The platform connects users with a variety of coaches and therapists and offers a personalized library of wellness content, which includes meditations, stories, and cognitive behavioral therapy (CBT).\n\nSources:\n- StrategistHub-Portfolio.pdf\n- Company Profile.pdf\n- PitchDeck.pdf'}


In [14]:
response = handle_query(type('obj', (object,), {'query': 'What are the major services of Strategisthub?'}))
print(response)

I am in the tool...
Got some data...
Re-Ranking the results...
{'response': 'StrategistHub offers a range of major services focused on helping businesses build scalable and high-performance digital solutions. Here are the key services:\n\n1. **Custom Software Development**: Tailored software solutions designed to meet specific business needs.\n\n2. **AI-Driven Solutions**: Integration of artificial intelligence to enhance operations and user experiences.\n\n3. **Advanced System Architecture**: Expertise in creating robust and efficient system architectures.\n\n4. **Engineering Excellence**: Providing highly skilled software engineers to drive digital transformation.\n\n5. **Consultation and Strategy**: Collaborating with clients to ensure solutions align with business goals and deliver measurable impact.\n\n6. **Support for Startups and Enterprises**: Services catered to both tech startups and large enterprises, addressing complex challenges across various industries.\n\nStrategistHub 